In [6]:
import pandas as pd
import math
from datetime import datetime, timezone
import pytz
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By

### Define dictionary for converting month and quarter data to date in DD-MM-YYYY format

In [2]:
month_dict = {'Jan' : '01-01', 'Feb' : '01-02', 'Mar' : '01-03', 'Apr' : '01-04', 'May' : '01-05', 'Jun': '01-06',
'Jul' : '01-07', 'Aug' : '01-08', 'Sep' : '01-09', 'Oct' : '01-10', 'Nov' : '01-11', 'Dec' : '01-12'}
qtr_dict = {'Q1' : '01-01', 'Q2' : '01-04', 'Q3' : '01-07', 'Q4' : '01-10'}

In [9]:
eastern_tz = pytz.timezone('US/Eastern')
utc_now = datetime.now(pytz.utc)
eastern_now = utc_now.astimezone(eastern_tz)
# current_date = datetime.now(timezone.utc).strftime('%d-%m-%Y')
current_date = eastern_now.date().strftime('%d-%m-%Y')

### Function for scraping website data

In [4]:
def scrape_data(url):
    options = Options()
    options.add_argument("--incognito")
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.51")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    return driver

### Function for getting date in 'DD-MM-YYYY' format

In [5]:
def get_date(date_string):
    if date_string[:3] in month_dict.keys():
        date_val =month_dict[date_string[:3]] + '-' + date_string[-5:-1]
    else:
        date_val =qtr_dict[date_string[:2]] + '-' + date_string[-5:-1]
    return date_val

### Load Feature data with URLs and HTML Elements to Dataframe

In [6]:
feature_df = pd.read_csv('Feature_list.csv')

### Extract website data along with error URLs if any

In [7]:
def get_attributes(df):
    values = {}
    dates = {}
    error_urls=[]
    for index, row in df.iterrows():
        try:
            print(row['Feature'])
            driver = scrape_data(row['URL'])
            try:
                value_element = driver.find_elements(By.XPATH, f'//div[@class="{row["div_element"]}"]')[0]
            except StaleElementReferenceException:
                driver.refresh()
                value_element = WebDriverWait(driver, 10).until(\
                EC.presence_of_all_elements_located((By.XPATH, f'//div[@class="{row["div_element"]}"]')))[0]
            if not isinstance(row['snap_element_value'],str) and math.isnan(row['snap_element_value']):
                values[row['Feature']] = value_element.text.replace('$', '')
            else:
                values[row['Feature']] = value_element.find_element(By.XPATH, f'.//span[@class="{row["snap_element_value"]}"]').text.replace('$', '')
            if not isinstance(row['snap_element_max_date'],str) and math.isnan(row['snap_element_max_date']):
                dates[row['Feature']] = current_date
            else:
                try:
                    date_element = driver.find_elements(By.XPATH, f'//div[@class="{row["div_element"]}"]')[0]
                    date_string = date_element.find_element(By.XPATH, f'.//span[@class="{row["snap_element_max_date"]}"]').text
                except StaleElementReferenceException:
                    driver.refresh()
                    date_element = WebDriverWait(driver, 10).until(\
                        EC.presence_of_all_elements_located((By.XPATH, f'//div[@class="{row["div_element"]}"]')))[0]
                    date_string = date_element.find_element(By.XPATH, f'.//span[@class="{row["snap_element_max_date"]}"]').text
                dates[row['Feature']] =  get_date(date_string)
        except Exception as e:
            print(e)
            error_urls.append(row['URL']) 
    return values, dates, error_urls

In [42]:
res_values, res_dates, error_urls = get_attributes(feature_df)

UNRATE
CPI
GDP
FED_GRANTS
S&P500
GOLD_PRICE
CRUDE_PRICE


In [65]:
data_df = pd.read_csv('exog_variables.csv', dayfirst=True, index_col=0)
data_df.loc[len(data_df), 'DATE'] = current_date
for key, value in res_dates.items():
    if pd.to_datetime(value, dayfirst=True) < pd.to_datetime(data_df['DATE'].iloc[-1], dayfirst=True):
        selected = pd.to_datetime(data_df['DATE'], dayfirst=True) >= pd.to_datetime(value, dayfirst=True)
        data_df.loc[selected, key] = res_values[key]      
    else:
        data_df.loc[len(data_df)-1, key] = res_values[key]

### Create output dataframe with ffill for missing values for any features

In [66]:
data_df['DATE'] = pd.to_datetime(data_df['DATE'], dayfirst=True)
df_daily = data_df.set_index('DATE').resample('D').asfreq()
df_daily.reset_index(inplace=True)
df_daily.loc[:, data_df.columns != 'COVID_STRINGENCY_INDEX'] = df_daily.loc[:, data_df.columns != 'COVID_STRINGENCY_INDEX'].fillna(method='ffill')

### Export the data

In [ ]:
df_daily.to_csv('exog_variables.csv')

In [30]:
json_dict = {}
json_dict['Update_date'] = current_date
json_dict['Error_URLs'] = error_urls

In [31]:
with open('run_summary.json', 'w') as f:
    json.dump(json_dict, f, indent=4)